I put the table of all result at first.

but you can see each model accuracy and structure in this Notebook.

## Tabel of models without Remove Hash or StopWord 

| model_name | model_summary | model_parameter | model_accuracy_test |
| :-: | :- | :-: | :-: |
|Base_Model | Embedding(300)-Dense(10)-Dense(5)|  9,171,217 | 0.6936 |
|CNN_model  | Embedding(300)-3*Conv1D(128)-Dense(128)-Dens(5) |14,119,981| 0.6479 |
|GRU_with_Regulaziation | Embedding(300)-2*GRU(128,recurent_dropout)-Dense(5)| 14,011,437 |  0.7889 |
|GRU_with_f1_and_Regulaization | Embedding(300)-2*GRU(128,recurent_dropout)-Dense(5)|14,011,437 | 0.7780 |
|LSTM_with_f1_and_Regulaization |  Embedding(300)-2*LST(128,recurent_dropout)-Dense(32)-Dense(5) | 14,102,125 | 0.7546|
|LSTM_32_Node | Embedding(300)-2*LST(32,recurent_dropout)-Dense(32)-Dense(5)|13,798,765 |0.7181|
|LSTM_Bidarictional | Embedding(300)-Bid(LSTM(64))-Dense(64)-Dense(5) | 13,834,093 | 0.7275 |
|...|...|...|...|
|Ensamble_model | GRU_with_Regulaziation + GRU_with_f1_and_Regulaization +LSTM_with_f1_and_Regulaization | None | 0.8071 | 

## Tabel of models with Remove Hash and StopWord 

| model_name | model_summary | model_parameter | model_accuracy_test |
| :-: | :- | :-: | :-: |
|GRU_with_Clan_StopWords | Embedding(300)-2*GRU(128,recurent_dropout)-Dense(5)|13,468,177| 0.7290|
|LSTM_with_Clan_StopWords_f1_score |Embedding(300)-2*LSTM(128,recurent_dropout)-Dense(5)|13,468,177| 0.7560|
|LSTM_64_With_CleanHash | Embedding(200)-Bidirectional(LSTM(128))-Dense(64)-Dense(5)|6,938,861|0.7845|
|...|...|...|...|

## Tabel of models with Remove just Hash (#)

| model_name | model_summary | model_parameter | model_accuracy_test |
| :-: | :- | :-: | :-: |
|GRU_with_clean_Hash|Embedding(300)-2*GRU(128,recurent_dropout)-Dense(5)|14,014,437|0.8006|
|LSTM_with_clean_Hash| Embedding(300) -2*LSTM(128)-Dense(5)|14,101,477|0.7886|
|GRU_Learning_scheduler_with_mask| Embedding(300)-2GRU(128)-Dense(5) | 14,014,437 | 0.7921|
|...|...|...|...|
|Ensamble_model| GRU_with_clean_Hash+LSTM_with_clean_Hash+GRU_Learning_scheduler_with_mask|None|0.8172|

## Tabel of Binary_classification models

| model_name | model_summary | model_parameter | model_accuracy_test |
| :-: | :- | :-: | :-: |
|Binary_classifiction|Embedding(300)-2*LSTM(128)-Dense(1)|12,506,761|0.8927
|Binary_classifiction_2|Embedding(300)-LSTM(128)-Dense(32)-Dense(1)|12,379,209|0.8949
|Binary_classifiction_with_transfer_learning |Embedding(300)-2*GRU(128)-Dense(1)|14,013,921|0.8448


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import seaborn as sn
import re

import tensorflow as tf 
import tensorflow.keras as keras 
import tensorflow_addons as tfa

import sklearn.model_selection as model_selection
import sklearn.metrics as metrics_model
import sklearn.linear_model as linear_model

In [7]:
f1=tfa.metrics.F1Score(5)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_set=pd.read_csv('/content/drive/MyDrive/train_df.csv')
test_set=pd.read_csv('/content/drive/MyDrive/test_df.csv')

X_train=train_set['clean_text'].values
X_test=test_set['clean_text'].values
y_train=train_set['NomericSentiment'].values
y_test=test_set['NomericSentiment'].values

X_train,X_val,y_train,y_val=model_selection.train_test_split(X_train,y_train,test_size=0.2,stratify=train_set['NomericSentiment'],random_state=42)

y_train=keras.utils.to_categorical(y_train,5)
y_test=keras.utils.to_categorical(y_test,5)
y_val=keras.utils.to_categorical(y_val,5)

tokinizer=keras.preprocessing.text.Tokenizer(num_words=30000)
tokinizer.fit_on_texts(train_set['clean_text'].values)

vocab_size=len(tokinizer.word_index)+1
print(vocab_size)

X_train=tokinizer.texts_to_sequences(X_train)
X_test=tokinizer.texts_to_sequences(X_test)
X_val=tokinizer.texts_to_sequences(X_val)
X_train_set=keras.preprocessing.sequence.pad_sequences(X_train,padding='post',maxlen=70)
X_test_set=keras.preprocessing.sequence.pad_sequences(X_test,padding='post',maxlen=70)
X_val_set=keras.preprocessing.sequence.pad_sequences(X_val,padding='post',maxlen=70)



45822


In [8]:
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS=stopwords.words('english')

def remove_stop_word_less_than_3_charecter(data):
  min_stop=[w for w in STOPWORDS if len(w)<=3]
  word=data.split()
  res=[w for w in word if w not in min_stop]
  return ' '.join(res)

def remove_hash(data):
  words=data.split()
  res=[w for w in words if '#' not in w]
  return ' '.join(res)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
import re
def remove_just_Hash(data):
  data=re.sub('#','',data)
  return data

Base model is DenseNet With Embedding

In [ ]:
Base_model=keras.models.load_model('/content/drive/MyDrive/Model/DenseNet_with_Embedding.h5')

In [ ]:
loss,acc=Base_model.evaluate(X_train_set,y_train)
print("Accuracy : {} ".format(acc))
loss,acc=Base_model.evaluate(X_test_set,y_test)
print("Accuracy : {} ".format(acc))


823/823 [==============================] - 1s 2ms/step - loss: 0.5685 - accuracy: 0.8180
Accuracy : 0.8179953694343567 
258/258 [==============================] - 0s 2ms/step - loss: 0.8438 - accuracy: 0.6936
Accuracy : 0.6936444044113159 


now late see the model we train for know 

In [ ]:
CNN_model=keras.models.load_model('/content/drive/MyDrive/Model/CNN_with_Embedding.h5')
loss,acc=CNN_model.evaluate(X_train_set,y_train)
print("Accuracy : {} ".format(acc))
loss,acc=CNN_model.evaluate(X_test_set,y_test)
print("Accuracy : {} ".format(acc))


823/823 [==============================] - 10s 11ms/step - loss: 0.6858 - accuracy: 0.7337
Accuracy : 0.7336776852607727 
258/258 [==============================] - 3s 12ms/step - loss: 0.9087 - accuracy: 0.6480
Accuracy : 0.6479523777961731 


These model is do better in generlization 

In [ ]:
GRURNN_with_Regulaziation=keras.models.load_model('/content/drive/MyDrive/Model/GRURNN_with_Regulaziation.h5')
GRURNN_with_Regulaziation.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           13746600  
_________________________________________________________________
gru (GRU)                    (None, 70, 128)           165120    
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 14,011,437
Trainable params: 14,011,437
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss,acc=GRURNN_with_Regulaziation.evaluate(X_train_set,y_train)
print("Accuracy : {:.4f}".format(acc))
loss,acc=GRURNN_with_Regulaziation.evaluate(X_test_set,y_test)
print("Accuracy : {:.4f}".format(acc))


823/823 [==============================] - 39s 46ms/step - loss: 0.3226 - accuracy: 0.8925
Accuracy : 0.8925
258/258 [==============================] - 12s 47ms/step - loss: 0.6043 - accuracy: 0.7889
Accuracy : 0.7889


as we can see this model still have overfit but its do better on training set and test set

In [ ]:
GRU_with_f1_and_Regulaization=keras.models.load_model('/content/drive/MyDrive/Model/GRURNN_with_Regulaziation_with_f1Score.h5')
GRU_with_f1_and_Regulaization.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           13746600  
_________________________________________________________________
gru (GRU)                    (None, 70, 128)           165120    
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 14,011,437
Trainable params: 14,011,437
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss,acc,f1_score=GRU_with_f1_and_Regulaization.evaluate(X_train_set,y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=GRU_with_f1_and_Regulaization.evaluate(X_test_set,y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))

Accuracy : 0.8784,f1_score:[0.85155183 0.860328   0.8996927  0.9082192  0.90566593]
Accuracy : 0.7780,f1_score:[0.7471169 0.7454546 0.835452  0.8109867 0.7900147]


this model is Better becouse its generalize better

In [ ]:
LSTMRNN_with_Regulaziation_with_f1Score=keras.models.load_model('/content/drive/MyDrive/Model/LSTMRNN_with_Regulaziation_with_f1Score.h5')
LSTMRNN_with_Regulaziation_with_f1Score.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           13746600  
_________________________________________________________________
lstm (LSTM)                  (None, 70, 128)           219648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 14,102,125
Trainable params: 14,102,125
Non-trainable params: 0
____________________________________________

In [ ]:
loss,acc,f1_score=LSTMRNN_with_Regulaziation_with_f1Score.evaluate(X_train_set,y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=LSTMRNN_with_Regulaziation_with_f1Score.evaluate(X_test_set,y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))


Accuracy : 0.8343,f1_score:[0.8187207  0.8150789  0.88279355 0.8357736  0.8424234 ]
Accuracy : 0.7546,f1_score:[0.74985665 0.72254056 0.81523675 0.7460036  0.75910926]


this model is the best for now from respective of generalization

In [ ]:
LSTM_32_Node=keras.models.load_model('/content/drive/MyDrive/Model/LSTM_32_Node.h5')
LSTM_32_Node.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 300)           13746600  
_________________________________________________________________
lstm (LSTM)                  (None, 60, 32)            42624     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 13,798,765
Trainable params: 13,798,765
Non-trainable params: 0
____________________________________________

In [ ]:
loss,acc,f1_score=LSTM_32_Node.evaluate(X_train_set,y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=LSTM_32_Node.evaluate(X_test_set,y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))

Accuracy : 0.8519,f1_score:[0.858193   0.798273   0.8960197  0.92418855 0.7842494 ]
Accuracy : 0.7181,f1_score:[0.7043203  0.67195004 0.7912226  0.7644548  0.664422  ]


In [ ]:
LSTM_Bidarictional=keras.models.load_model('/content/drive/MyDrive/Model/LSTM__1_32_Node_with_Bidirectional.h5')
LSTM_Bidarictional.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 300)           13746600  
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 64)            85248     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 13,834,093
Trainable params: 13,834,093
Non-trainable params: 0
____________________________________________

In [ ]:
loss,acc,f1_score=LSTM_32_Node.evaluate(X_train_set[:,:-10],y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=LSTM_32_Node.evaluate(X_test_set[:,:-10],y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))

Accuracy : 0.8638,f1_score:[0.8537154  0.81803566 0.9135204  0.9183412  0.8339745 ]
Accuracy : 0.7275,f1_score:[0.7034396  0.680453   0.80510974 0.7665662  0.70974463]


as we can see the best 2 model is LSTM_with_f1_Score
and GRU_with_f1_score

 # Ensambel Model 

now I am use The Best Three model Without Remove (**#**) and see How This model well do on the test set

So now we gone use Ensamble model of the best model we get for this moment


In [ ]:
def get_ensamble_predict(models,X_test):
  res=np.zeros((X_test.shape[0],5))
  preds=[]
  for i,model in enumerate(models):
    print('working...')
    pred=model.predict(X_test)
    print('Done_{}'.format(i+1))
    preds.append(pred)
  res=(0.4*preds[0]+0.3*preds[1]+0.3*preds[2])
  return res

In [ ]:
models=[GRURNN_with_Regulaziation,LSTMRNN_with_Regulaziation_with_f1Score,GRU_with_f1_and_Regulaization]

y_pred_train=get_ensamble_predict(models,X_train_set)
print('...............\n Train_done \n...............')
y_pred_test=get_ensamble_predict(models,X_test_set)


working...
Done_1
working...
Done_2
working...
Done_3
...............
 Train_done 
...............
working...
Done_1
working...
Done_2
working...
Done_3


In [ ]:
acc=metrics_model.accuracy_score(np.argmax(y_train,axis=1),np.argmax(y_pred_train,axis=1))
f1_score=metrics_model.f1_score(np.argmax(y_train,axis=1),np.argmax(y_pred_train,axis=1),average=None)
print("Training Accureacy : {:.4f}, f1_score : {}".format(acc,f1_score))

acc=metrics_model.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_test,axis=1))
f1_score=metrics_model.f1_score(np.argmax(y_test,axis=1),np.argmax(y_pred_test,axis=1),average=None)
print("Testing Accureacy : {:.4f}, f1_score : {}".format(acc,f1_score))

Training Accureacy : 0.8994, f1_score : [0.88330833 0.88634772 0.90365521 0.92577985 0.92132565]
Testing Accureacy : 0.8071, f1_score : [0.79187192 0.77144274 0.85012285 0.83578621 0.8175389 ]


as we can see the Ensemble model generalize better than each model alone 
and do as better as the model after we clean the (#) we try this on the last two model that we train after we remove the (#)

In [ ]:
LSTM__1_32_Node_with_Bidirectional_200d=keras.models.load_model('/content/drive/MyDrive/Model/LSTM__1_32_Node_with_Bidirectional_200d.h5')
LSTM__1_32_Node_with_Bidirectional_200d.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 200)           9164400   
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 64)            59648     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                650       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
Total params: 9,224,753
Trainable params: 9,224,753
Non-trainable params: 0
______________________________________________

In [ ]:
loss,acc,f1_score=LSTM__1_32_Node_with_Bidirectional_200d.evaluate(X_train_set[:,:-10],y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=LSTM__1_32_Node_with_Bidirectional_200d.evaluate(X_test_set[:,:-10],y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))

Accuracy : 0.8903,f1_score:[0.88749325 0.8644599  0.9123442  0.9021647  0.901139  ]
Accuracy : 0.7628,f1_score:[0.7458634  0.7206419  0.839327   0.76967686 0.76486737]


This model use embedding of 200D but it worse than befor

# remove Stop Word and Hash
now lets compare the model after we clean the stop word and Hash

In [ ]:
GRURNN_with_Regulaziation_with_f1Score_and_Clan_StopWords=keras.models.load_model('/content/drive/MyDrive/Model/GRURNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.h5')
GRURNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 300)           13116300  
_________________________________________________________________
lstm (LSTM)                  (None, 50, 128)           219648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 13,468,177
Trainable params: 13,468,177
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_set=pd.read_csv('/content/drive/MyDrive/train_df.csv')
test_set=pd.read_csv('/content/drive/MyDrive/test_df.csv')

train_set['clean_text_1']=train_set['clean_text'].apply(remove_stop_word_less_than_3_charecter)
test_set['clean_text_1']=test_set['clean_text'].apply(remove_stop_word_less_than_3_charecter)

X_train=train_set['clean_text_1'].values
X_test=test_set['clean_text_1'].values
y_train=train_set['NomericSentiment'].values
y_test=test_set['NomericSentiment'].values

X_train,X_val,y_train,y_val=model_selection.train_test_split(X_train,y_train,test_size=0.2,stratify=train_set['NomericSentiment'],random_state=42)

y_train=keras.utils.to_categorical(y_train,5)
y_test=keras.utils.to_categorical(y_test,5)
y_val=keras.utils.to_categorical(y_val,5)

tokinizer=keras.preprocessing.text.Tokenizer(num_words=30000)
tokinizer.fit_on_texts(train_set['clean_text_1'].values)

vocab_size=len(tokinizer.word_index)+1
print(vocab_size)

X_train=tokinizer.texts_to_sequences(X_train)
X_test=tokinizer.texts_to_sequences(X_test)
X_val=tokinizer.texts_to_sequences(X_val)
X_train_set=keras.preprocessing.sequence.pad_sequences(X_train,padding='post',maxlen=70)
X_test_set=keras.preprocessing.sequence.pad_sequences(X_test,padding='post',maxlen=70)
X_val_set=keras.preprocessing.sequence.pad_sequences(X_val,padding='post',maxlen=70)



43721


In [ ]:
loss,acc,f1_score=GRURNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.evaluate(X_train_set[:,:-20],y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=GRURNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.evaluate(X_test_set[:,:-20],y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))

Accuracy : 0.8178,f1_score:[0.8254593  0.77413213 0.86163926 0.8694514  0.7547939 ]
Accuracy : 0.7290,f1_score:[0.7260726 0.6829605 0.8026801 0.7845734 0.6426496]


we cant be sure about that but if we dont remove stop word i think its do better

In [ ]:
LSTMRNN_with_Regulaziation_with_f1Score_and_Clan_StopWords=keras.models.load_model('/content/drive/MyDrive/Model/LSTMRNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.h5')
LSTMRNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 300)           13116300  
_________________________________________________________________
lstm (LSTM)                  (None, 50, 128)           219648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 13,468,177
Trainable params: 13,468,177
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss,acc,f1_score=LSTMRNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.evaluate(X_train_set[:,:-20],y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=LSTMRNN_with_Regulaziation_with_f1Score_and_Clan_StopWords.evaluate(X_test_set[:,:-20],y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))

Accuracy : 0.8595,f1_score:[0.8470087  0.8264884  0.8852916  0.8906322  0.87286097]
Accuracy : 0.7560,f1_score:[0.73406965 0.69856703 0.82241726 0.79852396 0.76313365]


as we can see remove **stop word** its not good for this data as we expected in the first part

# remove Hash
 now lets try with removing hash

In [ ]:
train_set=pd.read_csv('/content/drive/MyDrive/train_df.csv')
test_set=pd.read_csv('/content/drive/MyDrive/test_df.csv')

train_set['clean_text_1']=train_set['clean_text'].apply(remove_hash)
test_set['clean_text_1']=test_set['clean_text'].apply(remove_hash)

X_train=train_set['clean_text_1'].values
X_test=test_set['clean_text_1'].values
y_train=train_set['NomericSentiment'].values
y_test=test_set['NomericSentiment'].values

X_train,X_val,y_train,y_val=model_selection.train_test_split(X_train,y_train,test_size=0.2,stratify=train_set['NomericSentiment'],random_state=42)

y_train=keras.utils.to_categorical(y_train,5)
y_test=keras.utils.to_categorical(y_test,5)
y_val=keras.utils.to_categorical(y_val,5)

tokinizer=keras.preprocessing.text.Tokenizer(num_words=30000)
tokinizer.fit_on_texts(train_set['clean_text_1'].values)

vocab_size=len(tokinizer.word_index)+1
print(vocab_size)

X_train=tokinizer.texts_to_sequences(X_train)
X_test=tokinizer.texts_to_sequences(X_test)
X_val=tokinizer.texts_to_sequences(X_val)
X_train_set=keras.preprocessing.sequence.pad_sequences(X_train,padding='post',maxlen=70)
X_test_set=keras.preprocessing.sequence.pad_sequences(X_test,padding='post',maxlen=70)
X_val_set=keras.preprocessing.sequence.pad_sequences(X_val,padding='post',maxlen=70)



33973


In [ ]:
LSTM_64_With_CleanHash=keras.models.load_model('/content/drive/MyDrive/Model/LSTM__64_Node_with_Bidirectional_200d_with_cleanHash.h5')
LSTM_64_With_CleanHash.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 200)           6794600   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               135680    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 6,938,861
Trainable params: 6,938,861
Non-trainable params: 0
______________________________________________

In [ ]:
loss,acc,f1_score=LSTM_64_With_CleanHash.evaluate(X_train_set[:,:-10],y_train,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))
loss,acc,f1_score=LSTM_64_With_CleanHash.evaluate(X_test_set[:,:-10],y_test,verbose=False)
print("Accuracy : {:.4f},f1_score:{}".format(acc,f1_score))

Accuracy : 0.9121,f1_score:[0.9005755  0.89599013 0.9268921  0.93551445 0.9193739 ]
Accuracy : 0.7845,f1_score:[0.76413745 0.7493846  0.8503938  0.80541587 0.78243715]


This model is do better on the training and test but still have overfit and The first model we have train do better than this model on test set

In [ ]:
import re
def remove_just_Hash(data):
  data=re.sub('#','',data)
  return data

now we see The model after we remove just (**#**) wich get the better result

In [21]:
train_set=pd.read_csv('/content/drive/MyDrive/train_df.csv')
test_set=pd.read_csv('/content/drive/MyDrive/test_df.csv')

train_set['clean_text_1']=train_set['clean_text'].apply(remove_just_Hash)
test_set['clean_text_1']=test_set['clean_text'].apply(remove_just_Hash)

X_train=train_set['clean_text_1'].values
X_test=test_set['clean_text_1'].values
y_train=train_set['NomericSentiment'].values
y_test=test_set['NomericSentiment'].values

X_train,X_val,y_train,y_val=model_selection.train_test_split(X_train,y_train,test_size=0.2,stratify=train_set['NomericSentiment'],random_state=42)

y_train=keras.utils.to_categorical(y_train,5)
y_test=keras.utils.to_categorical(y_test,5)
y_val=keras.utils.to_categorical(y_val,5)

tokinizer=keras.preprocessing.text.Tokenizer(num_words=30000)
tokinizer.fit_on_texts(train_set['clean_text_1'].values)

vocab_size=len(tokinizer.word_index)+1
print(vocab_size)

X_train=tokinizer.texts_to_sequences(X_train)
X_test=tokinizer.texts_to_sequences(X_test)
X_val=tokinizer.texts_to_sequences(X_val)
X_train_set=keras.preprocessing.sequence.pad_sequences(X_train,padding='post',maxlen=70)
X_test_set=keras.preprocessing.sequence.pad_sequences(X_test,padding='post',maxlen=70)
X_val_set=keras.preprocessing.sequence.pad_sequences(X_val,padding='post',maxlen=70)



45832


In [ ]:
GRURNN_with_Regulaization_And_clean_Hash=keras.models.load_model('/content/drive/MyDrive/GRU_with_Regulaization_And_clean_#.h5')
GRURNN_with_Regulaization_And_clean_Hash.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           13749600  
_________________________________________________________________
gru (GRU)                    (None, 70, 128)           165120    
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 14,014,437
Trainable params: 14,014,437
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss,acc=GRURNN_with_Regulaization_And_clean_Hash.evaluate(X_train_set,y_train)
print("Accuracy : {:.4f}".format(acc))
loss,acc=GRURNN_with_Regulaization_And_clean_Hash.evaluate(X_test_set,y_test)
print("Accuracy : {:.4f}".format(acc))

823/823 [==============================] - 38s 45ms/step - loss: 0.3170 - accuracy: 0.8972
Accuracy : 0.8972
258/258 [==============================] - 12s 45ms/step - loss: 0.5777 - accuracy: 0.8006
Accuracy : 0.8006


we can see that we get better result after we remove only the (**#**) charecter

we will traning another model but now with lstm cell 

In [ ]:
LSTM_WITh_Clean_Hash=keras.models.load_model('/content/drive/MyDrive/LSTM_with_Regulaization_And_clean_#.h5')
LSTM_WITh_Clean_Hash.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           13749600  
_________________________________________________________________
lstm (LSTM)                  (None, 70, 128)           219648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 14,101,477
Trainable params: 14,101,477
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, accuracy,f1_score = LSTM_WITh_Clean_Hash.evaluate(X_train_set, y_train, verbose=False)
print("Training Accuracy: {:.4f}, f1_score : {}".format(accuracy,f1_score))
loss, accuracy,f1_score = LSTM_WITh_Clean_Hash.evaluate(X_test_set, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}, f1_score : {}".format(accuracy,f1_score))

Training Accuracy: 0.9172, f1_score : [0.9086776  0.9009023  0.9106529  0.94649714 0.94011366]
Testing Accuracy:  0.7886, f1_score : [0.76172453 0.7564692  0.8420325  0.8135461  0.8081672 ]


This model do better on the traning but worse in the testing from The GRU model

In [ ]:
GRU_Learning_scheduler=keras.models.load_model('/content/drive/MyDrive/GRU_Learning_scheduler.h5')
GRU_Learning_scheduler.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 300)           13749600  
_________________________________________________________________
gru (GRU)                    (None, 60, 128)           165120    
_________________________________________________________________
gru_1 (GRU)                  (None, 60, 128)           99072     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 645       
Total params: 14,014,437
Trainable params: 14,014,437
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, accuracy = GRU_Learning_scheduler.evaluate(X_train_set, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = GRU_Learning_scheduler.evaluate(X_test_set, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9232
Testing Accuracy:  0.7921


In [22]:
GRU_Conv1D=keras.models.load_model('/content/drive/MyDrive/GRU_Conv1D.h5')
GRU_Conv1D.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 300)           13749600  
_________________________________________________________________
gru (GRU)                    (None, 60, 64)            70272     
_________________________________________________________________
dropout (Dropout)            (None, 60, 64)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 58, 32)            6176      
_________________________________________________________________
dropout_1 (Dropout)          (None, 58, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 56, 32)            3104      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0

In [24]:
loss,acc=GRU_Conv1D.evaluate(X_train_set[:,:-10],y_train)
print("Accuracy : {:.3f}".format(acc))
loss,acc=GRU_Conv1D.evaluate(X_test_set[:,:-10],y_test)
print("Accuracy : {:.3f}".format(acc))


823/823 [==============================] - 13s 15ms/step - loss: 0.3357 - accuracy: 0.9038
Accuracy : 0.904
258/258 [==============================] - 4s 15ms/step - loss: 0.6901 - accuracy: 0.7474
Accuracy : 0.747


# Ensambel Model 

we will do the same thing for model that we remove The (**#**) from it 

In [ ]:
models_2=[GRURNN_with_Regulaization_And_clean_Hash,LSTM_WITh_Clean_Hash,GRU_Learning_scheduler]

In [ ]:
y_pred_train=get_ensamble_predict(models_2,X_train_set)
print('.............\nTrain_done\n.............')
y_pred_test=get_ensamble_predict(models_2,X_test_set)


working...
Done_1
working...
Done_2
working...
Done_3
............./nTrain_done/n.............
working...
Done_1
working...
Done_2
working...
Done_3


In [ ]:
acc=metrics_model.accuracy_score(np.argmax(y_train,axis=1),np.argmax(y_pred_train,axis=1))
f1_score=metrics_model.f1_score(np.argmax(y_train,axis=1),np.argmax(y_pred_train,axis=1),average=None)
print("Training Accureacy : {:.4f}, f1_score : {}".format(acc,f1_score))

acc=metrics_model.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_test,axis=1))
f1_score=metrics_model.f1_score(np.argmax(y_test,axis=1),np.argmax(y_pred_test,axis=1),average=None)
print("Testing Accureacy : {:.4f}, f1_score : {}".format(acc,f1_score))

Training Accureacy : 0.9285, f1_score : [0.92358848 0.91557651 0.91679176 0.95529982 0.94776553]
Testing Accureacy : 0.8172, f1_score : [0.80225517 0.79148733 0.86571234 0.82828283 0.82194418]


we can see that the ensamble model on is better than using each model alone

so we chose this three model to make the ensamble model and we will predict with this three model.

but first we will retrain those model on the whole data you can see **Last_Training..** Notebook

# Binary Classification

In [10]:
train_set=pd.read_csv('/content/drive/MyDrive/train_df.csv')
test_set=pd.read_csv('/content/drive/MyDrive/test_df.csv')

train_set['clean_text_1']=train_set['clean_text'].apply(remove_just_Hash)
test_set['clean_text_1']=test_set['clean_text'].apply(remove_just_Hash)

dic={0:0,1:1,3:0,4:1}

train_set=train_set[train_set['NomericSentiment']!=2]
test_set=test_set[test_set['NomericSentiment']!=2]

train_set['label']=train_set['NomericSentiment'].apply(lambda x:dic[x])
test_set['label']=test_set['NomericSentiment'].apply(lambda x:dic[x])

X_train=train_set['clean_text_1'].values
X_test=test_set['clean_text_1'].values
y_train=train_set['label'].values
y_test=test_set['label'].values


X_train,X_val,y_train,y_val=model_selection.train_test_split(X_train,y_train,test_size=0.2,stratify=train_set['label'],random_state=42)

# y_train=keras.utils.to_categorical(y_train,2)
# y_test=keras.utils.to_categorical(y_test,2)
# y_val=keras.utils.to_categorical(y_val,2)
y_train=y_train.astype(np.uint8)
y_test=y_test.astype(np.uint8)
y_val=y_val.astype(np.uint8)

tokinizer=keras.preprocessing.text.Tokenizer(num_words=30000)
tokinizer.fit_on_texts(train_set['clean_text_1'].values)

vocab_size=len(tokinizer.word_index)+1
print(vocab_size)

X_train=tokinizer.texts_to_sequences(X_train)
X_test=tokinizer.texts_to_sequences(X_test)
X_val=tokinizer.texts_to_sequences(X_val)
X_train_set=keras.preprocessing.sequence.pad_sequences(X_train,padding='post',maxlen=70)
X_test_set=keras.preprocessing.sequence.pad_sequences(X_test,padding='post',maxlen=70)
X_val_set=keras.preprocessing.sequence.pad_sequences(X_val,padding='post',maxlen=70)


40518


In [11]:
Binary_calssification=keras.models.load_model('/content/drive/MyDrive/Binary_Classifaction.h5')
Binary_calssification.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           12155400  
_________________________________________________________________
lstm (LSTM)                  (None, 70, 128)           219648    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 12,506,761
Trainable params: 12,506,761
Non-trainable params: 0
_________________________________________________________________


In [12]:
loss,acc=Binary_calssification.evaluate(X_train_set,y_train,verbose=False)
print('Accuracy : {:.4f}'.format(acc))
loss,acc=Binary_calssification.evaluate(X_test_set,y_test,verbose=False)
print('Accuracy : {:.4f}'.format(acc))


Accuracy : 0.9644
Accuracy : 0.8927


In [15]:
Binary_Classifcation_2=keras.models.load_model('/content/drive/MyDrive/Binary_Classifaction_2.h5')
Binary_Classifcation_2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           12155400  
_________________________________________________________________
lstm (LSTM)                  (None, 70, 128)           219648    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 12,379,209
Trainable params: 12,379,209
Non-trainable params: 0
____________________________________________

In [17]:
loss,acc=Binary_Classifcation_2.evaluate(X_train_set,y_train,verbose=False)
print('Accuracy : {:.4f}'.format(acc))
loss,acc=Binary_Classifcation_2.evaluate(X_test_set,y_test,verbose=False)
print('Accuracy : {:.4f}'.format(acc))


Accuracy : 0.9649
Accuracy : 0.8949


In [19]:
Binary_calssifiaction_with_transfer_learning=keras.models.load_model('/content/drive/MyDrive/Binary_calssifiaction_with_transfer_learning.h5')
Binary_calssifiaction_with_transfer_learning.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 70, 300)           13749600  
_________________________________________________________________
gru (GRU)                    (None, 70, 128)           165120    
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 14,013,921
Trainable params: 14,013,921
Non-trainable params: 0
_________________________________________________________________


In [20]:
loss,acc=Binary_calssifiaction_with_transfer_learning.evaluate(X_train_set,y_train)
print("Accuracy : {:.4f}".format(acc))
loss,acc=Binary_calssifiaction_with_transfer_learning.evaluate(X_test_set,y_test)
print("Accuracy : {:.4f}".format(acc))


669/669 [==============================] - 30s 44ms/step - loss: 0.1274 - acc: 0.9614
Accuracy : 0.9614
210/210 [==============================] - 9s 44ms/step - loss: 0.3514 - acc: 0.8448
Accuracy : 0.8448
